In [1]:
import argparse
import torch
import torch.nn as nn
from torchvision import transforms
import numpy as np
from torchvision.datasets import MNIST, SVHN
from torch.utils.data import DataLoader
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
import random
import torchvision
import torchvision.models as models
import datetime
import sys
import sklearn
from torch.autograd import Function
import torch.nn.functional as F
from sklearn.metrics import f1_score
import os
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from torchvision import datasets

In [56]:
epochs = 10
lr = 0.03
momentum = 0.5
seed = 40
cuda = True
log_interval = 600
random = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
batch_size = 16
input_size = (128, 64)
aspect_ratio = [2, 1]
is_vit = False
if(torch.cuda.is_available()):
  device=torch.device('cuda')
  print("GPU")
else:
  device=torch.device('cpu')
  print('CPU')

GPU


In [3]:
def resize_image_with_aspect(image):
    w, h = image.size
    cut_h = h - aspect_ratio[1] * w / aspect_ratio[0]
    image = image.crop((0, cut_h / 2, w, h-(cut_h/2)))
    image = image.resize(input_size)
    return image

In [4]:
def calc_coeff(iter_num, high=1.0, low=0.0, alpha=10.0, max_iter=10000.0):
    return float(2.0 * (high - low) / (1.0 + np.exp(-alpha * iter_num / max_iter)) - (high - low) + low)

In [5]:
def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Conv2d') != -1 or classname.find('ConvTranspose2d') != -1:
        nn.init.kaiming_uniform_(m.weight)
        nn.init.zeros_(m.bias)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.zeros_(m.bias)
    elif classname.find('Linear') != -1:
        nn.init.xavier_normal_(m.weight)
        # nn.init.zeros_(m.bias)
        m.bias.data.fill_(0)

In [48]:
def Entropy(input_):
    epsilon = 1e-5
    entropy = -input_ * torch.log(input_ + epsilon)
    entropy = torch.sum(entropy, dim=1)
    return entropy


def grl_hook(coeff):
    def fun1(grad):
        return -coeff * grad.clone()

    return fun1


def CDAN(input_list, ad_net, entropy=None, coeff=None, random_layer=None):
    softmax_output = input_list[1].detach()
    feature = input_list[0]
    if random_layer is None:
        op_out = torch.bmm(softmax_output.unsqueeze(2), feature.unsqueeze(1))
        ad_out = ad_net(op_out.view(-1, softmax_output.size(1) * feature.size(1)))
    else:
        random_out = random_layer.forward([feature, softmax_output])
        ad_out = ad_net(random_out.view(-1, random_out.size(1)))
    batch_size = softmax_output.size(0) // 2
    dc_target = torch.from_numpy(np.array([[1]] * batch_size + [[0]] * batch_size)).float()
    if cuda:
        dc_target = dc_target.cuda()
    if entropy is not None:
        entropy.register_hook(grl_hook(coeff))
        entropy = 1.0 + torch.exp(-entropy)
        source_mask = torch.ones_like(entropy)
        source_mask[feature.size(0) // 2:] = 0
        source_weight = entropy * source_mask
        target_mask = torch.ones_like(entropy)
        target_mask[0:feature.size(0) // 2] = 0
        target_weight = entropy * target_mask
        weight = source_weight / torch.sum(source_weight).detach().item() + \
                 target_weight / torch.sum(target_weight).detach().item()
        l = nn.BCELoss(reduction='none')(ad_out, dc_target)
        return torch.sum(weight.view(-1, 1) * nn.BCELoss()(ad_out, dc_target)) / torch.sum(weight).detach().item()
    else:
        return nn.BCELoss()(ad_out, dc_target)


def mdd_loss(features, labels, left_weight=1, right_weight=1):
    softmax_out = nn.Softmax(dim=1)(features)
    batch_size = features.size(0)
    if float(batch_size) % 2 != 0:
        raise Exception('Incorrect batch size provided')

    batch_left = softmax_out[:int(0.5 * batch_size)]
    batch_right = softmax_out[int(0.5 * batch_size):]

    loss = torch.norm((batch_left - batch_right).abs(), 2, 1).sum() / float(batch_size)

    labels_left = labels[:int(0.5 * batch_size)]
    batch_left_loss = get_pari_loss1(labels_left, batch_left)

    labels_right = labels[int(0.5 * batch_size):]
    batch_right_loss = get_pari_loss1(labels_right, batch_right)
    return loss + left_weight * batch_left_loss + right_weight * batch_right_loss


def mdd_digit(features, labels, left_weight=1, right_weight=1, weight=1):
    softmax_out = nn.Softmax(dim=1)(features)
    batch_size = features.size(0)
    if float(batch_size) % 2 != 0:
        raise Exception('Incorrect batch size provided')

    batch_left = softmax_out[:int(0.5 * batch_size)]
    batch_right = softmax_out[int(0.5 * batch_size):]

    loss = torch.norm((batch_left - batch_right).abs(), 2, 1).sum() / float(batch_size)

    labels_left = labels[:int(0.5 * batch_size)]
    labels_left_left = labels_left[:int(0.25 * batch_size)]
    labels_left_right = labels_left[int(0.25 * batch_size):]

    batch_left_left = batch_left[:int(0.25 * batch_size)]
    batch_left_right = batch_left[int(0.25 * batch_size):]
    batch_left_loss = get_pair_loss(labels_left_left, labels_left_right, batch_left_left, batch_left_right)

    labels_right = labels[int(0.5 * batch_size):]
    labels_right_left = labels_right[:int(0.25 * batch_size)]
    labels_right_right = labels_right[int(0.25 * batch_size):]

    batch_right_left = batch_right[:int(0.25 * batch_size)]
    batch_right_right = batch_right[int(0.25 * batch_size):]
    batch_right_loss = get_pair_loss(labels_right_left, labels_right_right, batch_right_left, batch_right_right)

    return weight*loss + left_weight * batch_left_loss + right_weight * batch_right_loss

def get_pair_loss(labels_left, labels_right, features_left, features_right):
    loss = 0
    for i in range(len(labels_left)):
        if (labels_left[i] == labels_right[i]):
            loss += torch.norm((features_left[i] - features_right[i]).abs(), 2, 0).sum()
    return loss

def get_pari_loss1(labels, features):
    loss = 0
    count = 0
    for i in range(len(labels)):
        for j in range(i + 1, len(labels)):
            if (labels[i] == labels[j]):
                count += 1
                loss += torch.norm((features[i] - features[j]).abs(), 2, 0).sum()
    return loss / count

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.43335001, 0.43360192, 0.42602362), (0.28486016, 0.28320433, 0.28699529)),
])
flipTransform = transforms.RandomHorizontalFlip(p=1)
zoomTransform = transforms.RandomResizedCrop(input_size[:: -1], scale=(0.7, 1))
colorTransform = transforms.ColorJitter(brightness=(0.65, 0.9), contrast=(1.1, 1.35))

In [8]:
def generate_batches(df, train=True, validation=False):
    if train:
        labels = df['has_under_extrusion'].tolist()
    printers = df['printer_id'].unique().tolist()
    printer_domain_map = {printer_id: idx for idx, printer_id in enumerate(printers)}
    domains = [printer_domain_map[domain] for domain in df['printer_id'].tolist()]
    prints = df['print_id'].unique().tolist()
    print_domain_map = {print_id: idx for idx, print_id in enumerate(prints)}
    print_jobs = [print_domain_map[print] for print in df['print_id'].tolist()]
        
    image_paths = df['img_path'].tolist()
    current = 0
    while current < len(df):
        batch_images, batch_domains, batch_labels, batch_paths, batch_prints = [], [], [], [], []
        batch_idx = 0
        reserve_images, reserve_domains, reserve_labels, reserve_prints = [], [], [], []
        
        while batch_idx < batch_size:
            if current + batch_idx >= len(df):
                break
            
            image_path = image_paths[current + batch_idx]
            domain = domains[current + batch_idx]
            print_job = print_jobs[current + batch_idx]
            image_path = image_path.split('/')[-1]
            filename = '/home/shravandinakaran/data/'+image_path
            image = Image.open(filename)
            if is_vit:
                image = resize_to_square(image)
            else:
                image = resize_image_with_aspect(image)
            if train:
                label = labels[current + batch_idx]
                batch_labels.append(label)
            batch_domains.append(domain)
            batch_prints.append(print_job)
            batch_images.append(transform(image))
            batch_paths.append(image_path)
            batch_idx += 1
        current += batch_size
        if train:
            yield batch_images, np.array(batch_labels), np.array(batch_domains), np.array(batch_prints)
        else:
            yield batch_images, batch_paths

In [9]:
train_df = pd.read_csv('/home/shravandinakaran/data/train_.csv')
test_df = pd.read_csv('/home/shravandinakaran/data/test_.csv')
val_df = pd.read_csv('/home/shravandinakaran/data/val_.csv')
test_df = test_df.append(val_df)
rows = train_df.loc[train_df['print_id'].isin([1678419624, 1678445757, 1679265796, 1679221334, 1678589738])]
test_df = test_df.append(rows, ignore_index=True)
train_df.drop(rows.index, inplace=True)
print(len(test_df), len(train_df))
num_domains_train = len(train_df['printer_id'].unique().tolist())
num_domains_test = len(test_df['printer_id'].unique().tolist())
num_domains_total = len(set(train_df['printer_id'].unique().tolist() + test_df['printer_id'].unique().tolist()))

35900 40713


In [50]:
source_df = pd.read_csv('/home/shravandinakaran/data/source.csv')
target_df = pd.read_csv('/home/shravandinakaran/data/target.csv')
print(len(source_df), len(target_df))

30162 30693


In [10]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [11]:
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [12]:
class DTN(nn.Module):
    def __init__(self):
        super(DTN, self).__init__()
        self.conv_params = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm2d(64),
            nn.Dropout2d(0.1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm2d(128),
            nn.Dropout2d(0.3),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm2d(256),
            nn.Dropout2d(0.5),
            nn.ReLU()
        )

        self.fc_params = nn.Sequential(
            nn.Linear(256 * 8 * 16, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout()
        )

        self.classifier = nn.Linear(512, 2)
        self.__in_features = 512

    def forward(self, x):
        x = self.conv_params(x)
        x = x.view(x.size(0), -1)
        x = self.fc_params(x)
        y = self.classifier(x)
        return x, y

    def output_num(self):
        return self.__in_features

In [37]:
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.resnet = models.resnet18()
        self.resnet.fc = Identity()
        
        self.fc_params = nn.Sequential(
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout()
        )

        self.classifier = nn.Linear(512, 2)

    def forward(self, x):
        x = self.resnet(x)
        x = x.view(x.size(0), -1)
        x = self.fc_params(x)
        y = self.classifier(x)
        return x, y
        

In [24]:

class AdversarialNetwork(nn.Module):
    def __init__(self, in_feature, hidden_size):
        super(AdversarialNetwork, self).__init__()
        self.ad_layer1 = nn.Linear(in_feature, hidden_size)
        self.ad_layer2 = nn.Linear(hidden_size, hidden_size)
        self.ad_layer3 = nn.Linear(hidden_size, 1)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
        self.apply(init_weights)
        self.iter_num = 0
        self.alpha = 10
        self.low = 0.0
        self.high = 1.0
        self.max_iter = 10000.0

    def forward(self, x):
        if self.training:
            self.iter_num += 1
        coeff = calc_coeff(self.iter_num, self.high, self.low, self.alpha, self.max_iter)
        x = x * 1.0
        x.register_hook(grl_hook(coeff))
        x = self.ad_layer1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.ad_layer2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        y = self.ad_layer3(x)
        y = self.sigmoid(y)
        return y

    def output_num(self):
        return 1

    def get_parameters(self):
        return [{"params": self.parameters(), "lr_mult": 10, 'decay_mult': 2}]

In [63]:
def test(epoch, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target, _, _ in test_loader:
        data = torch.stack(data, dim=0, out=None).to(device)
        target = torch.from_numpy(target).to(device)
        feature, output = model(data)
        test_loss += nn.CrossEntropyLoss()(output, target).item()
        pred = output.data.cpu().max(1, keepdim=True)[1]
        correct += pred.eq(target.data.cpu().view_as(pred)).sum().item()
        
    test_loss /= len(target_df)
    acc = 100. * correct / len(target_df)
    log_str = 'epoch:{},Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(epoch,
                                                                                            test_loss, correct,
                                                                                            len(target_df),
                                                                                            acc)

    print(log_str)
    return acc

In [68]:
model = ResNet18() #DTN()
if cuda:
    model = model.cuda()
class_num = 2

if random:
    random_layer = network.RandomLayer([model.output_num(), class_num], 500)
    ad_net = AdversarialNetwork(500, 500)
    if cuda:
        random_layer.cuda()
else:
    random_layer = None
    ad_net = AdversarialNetwork(512 * class_num, 500)
if cuda:
    ad_net = ad_net.cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=0.0005, momentum=0.9)
optimizer_ad = optim.SGD(ad_net.parameters(), lr=lr, weight_decay=0.0005, momentum=0.9)

CELoss = nn.CrossEntropyLoss()
Softmax = nn.Softmax(dim=1)
mdd_weight = 1

best_model = model
best_acc = 0

In [ ]:
for epoch in range(1, epochs + 1):
    if epoch % 3 == 0:
        for param_group in optimizer.param_groups:
            param_group["lr"] = param_group["lr"] * 0.3
            
    source_df = source_df.sample(frac=1).reset_index(drop=True)
    target_df = target_df.sample(frac=1).reset_index(drop=True)
    source_generator = generate_batches(source_df)
    target_generator = generate_batches(target_df)
    
    model.train()
    len_source = len(source_df)
    len_target = len(target_df)
    num_iter = min(len_source // batch_size, len_target // batch_size)
    total_loss, correct = 0, 0
    for batch_idx in range(num_iter):
        data_source, label_source, _, _ = next(source_generator)
        data_source = torch.stack(data_source, dim=0, out=None).to(device)
        label_source = torch.from_numpy(label_source).to(device)
        
        data_target, label_target, _, _ = next(target_generator)
        data_target = torch.stack(data_target, dim=0, out=None).to(device)
        label_target = torch.from_numpy(label_target).to(device)
        
        optimizer.zero_grad()
        optimizer_ad.zero_grad()

        feature_source, output_source = model(data_source)
        feature_target, output_target = model(data_target)

        feature = torch.cat((feature_source, feature_target), 0)
        output = torch.cat((output_source, output_target), 0)

        labels_target_fake = torch.max(nn.Softmax(dim=1)(output_target), 1)[1]
        labels = torch.cat((label_source, labels_target_fake))

        loss = CELoss(output.narrow(0, 0, data_source.size(0)), label_source)

        softmax_output = Softmax(output)
        if epoch > 0:
            entropy = Entropy(softmax_output)
            loss += CDAN([feature, softmax_output], ad_net, entropy,
                                   calc_coeff(num_iter * (epoch - 0) + batch_idx), random_layer)

        mdd_loss = 0.0 #mdd_weight * mdd_loss(feature, labels)
        loss = loss + mdd_loss

        total_loss += loss.data
        
        pred = output_source.data.cpu().max(1, keepdim=True)[1]
        correct += pred.eq(label_source.data.cpu().view_as(pred)).sum().item()

        loss.backward()
        optimizer.step()
        if epoch > 0:
            optimizer_ad.step()
        if (batch_idx + epoch * num_iter) % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.3f}'.format(
                epoch, batch_idx * batch_size, num_iter * batch_size,
                       100. * batch_idx / num_iter, loss.item()))
    
    log_str = "train_loss:{:.3f}, accuracy:{:.3f}\n".format(total_loss / num_iter, 100*correct / len(source_df))
    print(log_str)
    
    target_generator = generate_batches(target_df)
    acc = test(epoch, model, target_generator)

Train Epoch: 1 [8240/30160 (27%)]	Loss: 0.673
Train Epoch: 1 [17840/30160 (59%)]	Loss: 0.683
Train Epoch: 1 [27440/30160 (91%)]	Loss: 0.705
train_loss:0.752, accuracy:98.873

epoch:1,Test set: Average loss: 0.3793, Accuracy: 12892/30693 (42.0031%)

Best Acc so far:  42.003062587560684
Train Epoch: 2 [6880/30160 (23%)]	Loss: 0.705
Train Epoch: 2 [16480/30160 (55%)]	Loss: 0.721
Train Epoch: 2 [26080/30160 (86%)]	Loss: 0.690
train_loss:0.701, accuracy:99.831

epoch:2,Test set: Average loss: 0.3626, Accuracy: 14374/30693 (46.8315%)

Best Acc so far:  46.83152510344378
Train Epoch: 3 [5520/30160 (18%)]	Loss: 0.703
Train Epoch: 3 [15120/30160 (50%)]	Loss: 0.679
Train Epoch: 3 [24720/30160 (82%)]	Loss: 0.714
train_loss:0.693, accuracy:99.910

epoch:3,Test set: Average loss: 0.3795, Accuracy: 12697/30693 (41.3677%)

Train Epoch: 4 [4160/30160 (14%)]	Loss: 0.720
Train Epoch: 4 [13760/30160 (46%)]	Loss: 0.690
Train Epoch: 4 [23360/30160 (77%)]	Loss: 0.684
train_loss:0.688, accuracy:99.980

epoch

In [64]:
target_generator = generate_batches(target_df)
acc = test(epoch, model, target_generator)

epoch:2,Test set: Average loss: 0.2358, Accuracy: 11900/30693 (38.7711%)

